# CNN (畳み込みニューラルネットワーク)

CNN(Convolutional Neural Network)、ConvNetなどと呼ばれる。
画像認識に対して高い精度を出す。人間を超えるモデルなどもある。
Convolution層(畳み込み層)、Pooling層（プーリング層)などの層が特徴的。

In [7]:
# ライブラリのインポート
import tensorflow as tf
import numpy as np

In [2]:
# MNISTデータのロード
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# ハイパーパラメータ
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

In [4]:
# ネットワークパラメータ
n_input     = 784  # MNISTの入力データ (height:28 x width:28の画像なので784)
n_classes = 10     # MNISTのクラス (0-9の数字なので10)
dropout    = 0.75 # ドロップアウトの率。75%のユニットは伝播する。25%はドロップアウトさせる

In [5]:
# グラフの入力を定義
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # ドロップアウトしない率を設定

In [6]:
# ノードを簡単に作成できるラッパーを作成
def conv2d(x, W, b, strides=1):
    # conv2d、バイアス、ReLUアクティベーションをひとまとめにしたもの
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # max_poolをラッピング
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [21]:
# モデルの作成
def conv_net(x, weights, biases, dropout):
    # 入力画像をreshapeする
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # 畳み込み層
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # プーリング層
    conv1 = maxpool2d(conv1, k=2)
    
    # 畳み込み層
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # プーリング層
    conv2 = maxpool2d(conv2, k=2)
    
    # Affine層（全結合層）
    # conv2の出力を全結合層に合わせるために、reshapeする
    affine1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    affine1 = tf.add(tf.matmul(affine1, weights['wd1']), biases['bd1'])
    affine1 = tf.nn.relu(affine1)
    
    # Dropout
    affine1 = tf.nn.dropout(affine1, dropout)
    
    # クラス分類
    out = tf.add(tf.matmul(affine1, weights['out']), biases['out'])
    return out

In [32]:
# 各層の重みとバイアスを設定
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # 全結合層 (7*7*64 inputs, 1024 outputs)
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# CNNモデルの作成
pred = conv_net(x, weights, biases, keep_prob)

# コスト関数とオプティマイザを定義
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# モデルの評価
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 変数の初期化
init = tf.initialize_all_variables()

In [31]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))

Iter 1280, Minibatch Loss= 24887.816406, Training Accuracy= 0.32031
Iter 2560, Minibatch Loss= 15019.813477, Training Accuracy= 0.49219
Iter 3840, Minibatch Loss= 10054.588867, Training Accuracy= 0.64062
Iter 5120, Minibatch Loss= 6875.897461, Training Accuracy= 0.68750
Iter 6400, Minibatch Loss= 5117.779297, Training Accuracy= 0.79688
Iter 7680, Minibatch Loss= 7011.356445, Training Accuracy= 0.75000
Iter 8960, Minibatch Loss= 3421.235107, Training Accuracy= 0.82812
Iter 10240, Minibatch Loss= 3843.274902, Training Accuracy= 0.82031
Iter 11520, Minibatch Loss= 1790.023682, Training Accuracy= 0.90625
Iter 12800, Minibatch Loss= 3610.965332, Training Accuracy= 0.84375
Iter 14080, Minibatch Loss= 1174.718018, Training Accuracy= 0.90625
Iter 15360, Minibatch Loss= 1182.849731, Training Accuracy= 0.92969
Iter 16640, Minibatch Loss= 2311.872559, Training Accuracy= 0.89062
Iter 17920, Minibatch Loss= 1767.276611, Training Accuracy= 0.89844
Iter 19200, Minibatch Loss= 1665.861938, Training Ac